# Setup

In [1]:
import sys

sys.path.append("..")

In [2]:
# Main Constants
from pathlib import Path

# Base path for testing - adjust as needed
BASE_PATH = Path("temp").resolve()  # Points to anthropic_agent root
print(BASE_PATH)

/Users/aurosoni/Documents/AI Forge/Repos/anthropic_agent/anthropic_agent/common_tools/temp


# Tool Tests

## Tool: Read File

In [3]:
from anthropic_agent.common_tools.read_file import ReadFileTool

# Create the read file tool
# Include .yaml in allowed extensions to read todo files
read_file_tool = ReadFileTool(base_path=BASE_PATH, allowed_extensions={".md", ".mmd", ".yaml"})
read_file = read_file_tool.get_tool()

In [4]:
print(read_file("README.md"))

[lines 1-100 of 102 in README.md]
# anthropic-agent

A small Python library for building **Anthropic-powered agents** with:
- **streaming** responses
- **tool calling** (client tools)
- **optional server tools** (passed through to Anthropic)
- **persistence** (filesystem or Postgres) and **file storage** (local or S3)

> **For sample agent configurations examples hanve a look at agent_test.ipynb**

The primary package is `anthropic_agent` (install name: `anthropic-agent`).

## AnthropicAgent features

- **Resumable sessions**: pass `agent_uuid` to reload `messages`, `container_id`, token counters, and file registry from the DB backend.
- **Streaming output**: stream formatted chunks to an `asyncio.Queue` (`formatter="xml"` or `formatter="raw"`).
- **Tool execution loop**: executes *client tools* locally when the model returns `stop_reason == "tool_use"`, then feeds tool results back to the model.
- **Server tools passthrough**: include Anthropic server tools configs (e.g. code executio

## Tool: List Dir

In [5]:
from anthropic_agent.common_tools.list_dir import ListDirTool

# Include .yaml in allowed extensions to list todo files
list_dir_tool = ListDirTool(base_path=BASE_PATH, allowed_extensions={".md", ".mmd", ".yaml"})
list_dir = list_dir_tool.get_tool()

In [6]:
print(list_dir("."))

temp/
   - test-agent-123/
      - plan_20260115_074238.md
      - plan_20260115_074249.md
      - todos.yaml
   - README.md
   - test_file.md


## Tool: Glob File Search

In [7]:
from anthropic_agent.common_tools.glob_file_search import GlobFileSearchTool

glob_search_tool = GlobFileSearchTool(base_path=BASE_PATH)
glob_file_search = glob_search_tool.get_tool()

In [8]:
print(glob_file_search("*.md"))

test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md
README.md
test_file.md


## Tool: Grep Search

In [9]:
from anthropic_agent.common_tools.grep_search import GrepSearchTool

grep_search_tool = GrepSearchTool(base_path=BASE_PATH)
grep_search = grep_search_tool.get_tool()

In [10]:
print(grep_search("agent/|Test"))

test-agent-123/plan_20260115_074249.md:
  7- 1. Set up PostgreSQL instance
  8- 2. Create migration scripts
  9: 3. <match>Test</match> data integrity
  10- 4. Switch production connection
README.md:
  7- - **persistence** (filesystem or Postgres) and **file storage** (local or S3)
  8- 
  9: > **For sample agent configurations examples hanve a look at agent_<match>test</match>.ipynb**
  10- 
  11- The primary package is `anthropic_agent` (install name: `anthropic-agent`).
  23- - **Files API + storage backends** (optional): detect generated `file_id`s, download via Anthropic Files API, and store via local filesystem or S3.
  24- 
  25: For a step-by-step diagram of the run loop, see `anthropic_<match>agent/</match>agent-flow.md`.
  26- 
  27- ## Requirements
  57- 
  58- Notes:
  59: - For local tool calling, pass `tools=[...]` (see `anthropic_<match>agent/</match>tools/sample_tools.py`).
  60- - Runs are persisted under `./data/` by default (agent config, conversation history, run lo

## Tool: Apply Patch

In [11]:
from anthropic_agent.common_tools.apply_patch import ApplyPatchTool

apply_patch_tool = ApplyPatchTool(base_path=BASE_PATH)
apply_patch = apply_patch_tool.get_tool()

In [12]:
from __future__ import annotations

import json
from pathlib import Path
from pprint import pprint

# We'll demonstrate ALL patch operation types on the same logical file.
# Because BASE_PATH is your "temp" folder, all paths below are relative to that.
DEMO_PATH = "patch_demo.md"
DEMO_MOVED_PATH = "patch_demo_moved.md"


def run_patch(title: str, patch_text: str, *, dry_run: bool = False) -> None:
    # print(f"\n--- {title} (dry_run={dry_run}) ---")
    pprint(json.loads(apply_patch(patch_text, dry_run=dry_run)))


# Ensure clean slate so "Add File" works
for p in [Path(BASE_PATH) / DEMO_PATH, Path(BASE_PATH) / DEMO_MOVED_PATH]:
    if p.exists():
        p.unlink()

print("Demo files prepared.")

Demo files prepared.


In [13]:
# 1) Add File
patch_add = """*** Begin Patch
*** Add File: patch_demo.md
+# Patch Demo
+
+Alpha
+Beta
+
+## Section
+- one
+- two
+
*** End Patch"""

run_patch("Add File", patch_add, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 9,
 'lines_removed': 0,
 'op': 'add',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [14]:
# 2) Update File (simple hunk)
patch_update_simple = """*** Begin Patch
*** Update File: patch_demo.md
@@
 Alpha
-Beta
+Beta v2
*** End Patch"""

run_patch("Update File (simple)", patch_update_simple, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [15]:
# 3) Update File with a scope line (narrows matching to a subsection)
# NOTE: The scope line `@@ ## Section` anchors the search; don't repeat it as context
patch_update_scoped = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
 - one
-- two
+- two (updated)
*** End Patch"""

run_patch("Update File (scoped)", patch_update_scoped, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [16]:
# 4) Add a nested section so we can demo nested scopes
# Use EOF marker to append at end of file
patch_add_subsection = """*** Begin Patch
*** Update File: patch_demo.md
@@
 - two (updated)
+
+### Subsection
+note: hello
*** End of File
*** End Patch"""

run_patch("Update File (add subsection)", patch_add_subsection, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 3,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [17]:
# 5) Update File with nested scopes (class -> method style, but works for headings too)
# Nested scopes: @@ ## Section then @@ ### Subsection narrows to that subsection
patch_update_nested_scopes = """*** Begin Patch
*** Update File: patch_demo.md
@@ ## Section
@@ ### Subsection
-note: hello
+note: hello (edited)
*** End Patch"""

run_patch("Update File (nested scopes)", patch_update_nested_scopes, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 1,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [18]:
# 6) Update File targeting end-of-file (EOF marker)
# EOF marker ensures matching happens at the end of file
patch_update_eof = """*** Begin Patch
*** Update File: patch_demo.md
@@
 note: hello (edited)
+EOF line added
*** End of File
*** End Patch"""

run_patch("Update File (EOF marker)", patch_update_eof, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'op': 'update',
 'path': 'patch_demo.md',
 'status': 'ok'}


In [19]:
# 7) Move/Rename during an update
patch_move = """*** Begin Patch
*** Update File: patch_demo.md
*** Move to: patch_demo_moved.md
@@
 # Patch Demo
+# (moved)
*** End Patch"""

run_patch("Update File + Move to", patch_move, dry_run=False)

{'dry_run': False,
 'hunks_applied': 1,
 'lines_added': 1,
 'lines_removed': 0,
 'moved_from': 'patch_demo.md',
 'op': 'update',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [20]:
# 8) Delete File
patch_delete = """*** Begin Patch
*** Delete File: patch_demo_moved.md
*** End Patch"""

run_patch("Delete File", patch_delete, dry_run=False)

{'dry_run': False,
 'hunks_applied': 0,
 'lines_added': 0,
 'lines_removed': 14,
 'op': 'delete',
 'path': 'patch_demo_moved.md',
 'status': 'ok'}


In [21]:
# Optional: verify end state
print(list_dir("."))

temp/
   - test-agent-123/
      - plan_20260115_074238.md
      - plan_20260115_074249.md
      - todos.yaml
   - README.md
   - test_file.md


In [22]:
# Optional: inspect file content (should be gone after delete)
print(read_file("patch_demo_moved.md"))

Path does not exist: patch_demo_moved.md


## Tool : TODOs

In [23]:
from anthropic_agent.common_tools.todo_write import TodoWriteTool

# Create the todo tool with the same test agent UUID
todo_write = TodoWriteTool(base_path=BASE_PATH)
todo_write.set_agent_uuid("test-agent-123")  # Manually set for testing
todo_write = todo_write.get_tool()

In [24]:
# Create TODOs
# Note: Returns YAML-like structured format with status breakdown
result = todo_write(
    todos=[
        {"id": "review-pr", "content": "Review PR #42 for auth changes"},
        {"id": "update-docs", "content": "Update API documentation"},
        {"id": "fix-bug", "content": "Fix login timeout issue", "status": "in_progress"},
    ]
)
print(result)

status: ok
operation: replaced
path: test-agent-123/todos.yaml
summary:
  total: 3
  pending: 2
  in_progress: 1
  completed: 0
  cancelled: 0


In [25]:
# Change status of a todo, simultaneously creating a new todo
# Note: 'operation: updated' indicates merge mode was used
result = todo_write(
    merge=True,
    todos=[
        {"id": "fix-bug", "content": "Fix login timeout issue", "status": "completed"},
        {"id": "verify-pr", "content": "Verify PR #42 for auth changes", "status": "pending"},
    ]
)
print(result)

status: ok
operation: updated
path: test-agent-123/todos.yaml
summary:
  total: 4
  pending: 3
  in_progress: 0
  completed: 1
  cancelled: 0


In [26]:
# Status-only update (merge mode allows omitting content)
result = todo_write(
    merge=True,
    todos=[
        {"id": "review-pr", "status": "completed"},  # Just update status
    ]
)
print(result)

status: ok
operation: updated
path: test-agent-123/todos.yaml
summary:
  total: 4
  pending: 2
  in_progress: 0
  completed: 2
  cancelled: 0


In [27]:
# Cancel a todo
result = todo_write(
    merge=True,
    todos=[
        {"id": "update-docs", "status": "cancelled"},
    ]
)
print(result)

status: ok
operation: updated
path: test-agent-123/todos.yaml
summary:
  total: 4
  pending: 1
  in_progress: 0
  completed: 2
  cancelled: 1


In [28]:
# Verify todos file state
print(read_file("test-agent-123/todos.yaml"))

[lines 1-13 of 13 in test-agent-123/todos.yaml]
todos:
  - id: review-pr
    content: Review PR #42 for auth changes
    status: completed
  - id: update-docs
    content: Update API documentation
    status: cancelled
  - id: fix-bug
    content: Fix login timeout issue
    status: completed
  - id: verify-pr
    content: Verify PR #42 for auth changes
    status: pending



In [29]:
# Error case: Missing id field
result = todo_write(
    todos=[{"content": "No id provided"}]
)
print(result)

Error: Todo at index 0 is missing required 'id' field.


In [30]:
# Error case: Invalid status
result = todo_write(
    todos=[{"id": "bad-status", "content": "Test", "status": "invalid_status"}]
)
print(result)

Error: Todo 'bad-status' has invalid status 'invalid_status'. Valid statuses: cancelled, completed, in_progress, pending


In [31]:
# Error case: Empty todos list
result = todo_write(todos=[])
print(result)

Error: No todos provided.


## Tool : Create Plan

In [32]:
from anthropic_agent.common_tools.create_plan import CreatePlanTool

# Create the plan tool with the same test agent UUID
create_plan_tool = CreatePlanTool(base_path=BASE_PATH)
create_plan_tool.set_agent_uuid("test-agent-123")  # Same agent as todo_write
create_plan = create_plan_tool.get_tool()

In [33]:
# Basic plan creation with name parameter
# Note: Returns YAML-like structured format with status breakdown
plan_content = """# API Refactoring Plan

## Overview
Refactor the REST API to improve performance and maintainability.

## Steps
1. Audit current endpoints
2. Identify bottlenecks
3. Implement caching layer
4. Update documentation
"""

result = create_plan(plan=plan_content, name="api-refactor")
print(result)

status: ok
plan_path: test-agent-123/api_refactor_20260115_100908.md
todos_updated: false


In [34]:
# Plan with name and todos - creates plan AND appends todos to shared todos.yaml
# Note: Returns structured response with todos_summary showing status breakdown
plan_with_todos = """# Database Migration Plan

## Overview
Migrate from SQLite to PostgreSQL for production.

## Steps
1. Set up PostgreSQL instance
2. Create migration scripts
3. Test data integrity
4. Switch production connection
"""

todos_for_plan = [
    {"id": "setup-postgres", "content": "Set up PostgreSQL instance"},
    {"id": "create-migrations", "content": "Create migration scripts"},
    {"id": "test-integrity", "content": "Test data integrity after migration"},
]

result = create_plan(plan=plan_with_todos, name="db-migration", todos=todos_for_plan)
print(result)

status: ok
plan_path: test-agent-123/db_migration_20260115_100908.md
todos_updated: true
todos_path: test-agent-123/todos.yaml
todos_summary:
  total: 7
  pending: 4
  in_progress: 0
  completed: 2
  cancelled: 1


In [35]:
# Verify todos were appended (should include both original and plan-created todos)
print(read_file("test-agent-123/todos.yaml"))

[lines 1-22 of 22 in test-agent-123/todos.yaml]
todos:
  - id: review-pr
    content: Review PR #42 for auth changes
    status: completed
  - id: update-docs
    content: Update API documentation
    status: cancelled
  - id: fix-bug
    content: Fix login timeout issue
    status: completed
  - id: verify-pr
    content: Verify PR #42 for auth changes
    status: pending
  - id: setup-postgres
    content: Set up PostgreSQL instance
    status: pending
  - id: create-migrations
    content: Create migration scripts
    status: pending
  - id: test-integrity
    content: Test data integrity after migration
    status: pending



In [36]:
# List all files in agent directory (should show plans and todos.yaml)
print(list_dir("test-agent-123"))

test-agent-123/
   - api_refactor_20260115_100908.md
   - db_migration_20260115_100908.md
   - plan_20260115_074238.md
   - plan_20260115_074249.md
   - todos.yaml


In [37]:
# Read back a created plan file (use glob to find it first)
plan_files = glob_file_search("test-agent-123/plan_*.md")
print("Found plan files:")
print(plan_files)
if plan_files:
    first_plan = plan_files.strip().split("\n")[0]
    print(f"\n--- Contents of {first_plan} ---")
    print(read_file(first_plan))

Found plan files:
test-agent-123/plan_20260115_074249.md
test-agent-123/plan_20260115_074238.md

--- Contents of test-agent-123/plan_20260115_074249.md ---
[lines 1-10 of 10 in test-agent-123/plan_20260115_074249.md]
# Database Migration Plan

## Overview
Migrate from SQLite to PostgreSQL for production.

## Steps
1. Set up PostgreSQL instance
2. Create migration scripts
3. Test data integrity
4. Switch production connection



In [38]:
# Plan with name containing special characters (will be sanitized)
# "API v2 Upgrade!" becomes "API_v2_Upgrade" in the filename
result = create_plan(
    plan="# API v2 Upgrade\n\nUpgrade notes here.",
    name="API v2 Upgrade!"  # Spaces and special chars get sanitized
)
print(result)

status: ok
plan_path: test-agent-123/API_v2_Upgrade_20260115_100908.md
todos_updated: false


In [39]:
# Error case: Empty plan content
result = create_plan(plan="")
print(result)

Error: Plan content cannot be empty.


In [40]:
# Error case: Plan with invalid todos (missing content)
result = create_plan(
    plan="# Valid Plan\nSome content here.",
    todos=[{"id": "missing-content"}]  # Missing required 'content' field
)
print(result)

Error: Todo 'missing-content' is missing required 'content' field.
